# 如何查看和更新​​过去的图状态

!!!提示“先决条件”

本指南假设您熟悉以下概念：

* [时间旅行](/langgraphjs/concepts/time-travel)
* [断点](/langgraphjs/concepts/breakpoints)
* [LangGraph 术语表](/langgraphjs/concepts/low_level)

一旦你开始[检查点](./persistence.ipynb)你的图表，你可以轻松地
**获取**或**更新**在任何时间点的代理状态。这允许
有几件事：

1. 您可以在中断期间向用户显示一个状态，让他们接受
行动。
2. 您可以**倒回**图表以重现或避免问题。
3. 您可以**修改**状态以将代理嵌入到更大的系统中，或者
让用户更好地控制其动作。

用于此功能的关键方法是：

- [getState](/langgraphjs/reference/classes/langgraph_pregel.Pregel.html#getState):
从目标配置中获取值
- [updateState](/langgraphjs/reference/classes/langgraph_pregel.Pregel.html#updateState):
将给定值应用于目标状态

**注意：** 这需要传入一个检查指针。

<!-- 示例：
```javascript
TODO
...
``` -->
这适用于
[StateGraph](/langgraphjs/reference/classes/langgraph.StateGraph.html)
及其所有子类，例如
[MessageGraph](/langgraphjs/reference/classes/langgraph.MessageGraph.html)。

下面是一个例子。

<div class="警告提示">
<p class="admonition-title">注意</p>
<p>
在本指南中，我们将从头开始创建透明（但冗长）的代理。您可以使用 <code>createReactAgent(model, tools=tool, checkpointer=checkpointer)</code> (<a href="/langgraphjs/reference/functions/langgraph_prebuilt.createReactAgent.html">API doc</a>) 构造函数完成类似的功能。如果你习惯了 LangChain 的 <a href="https://js.langchain.com/docs/how_to/agent_executor">AgentExecutor</a> 类，这可能更合适。
</p>
</div>

## 设置

本指南将使用 OpenAI 的 GPT-4o 模型。我们可以选择设置 API 密钥
对于 [LangSmith 追踪](https://smith.langchain.com/)，这将为我们提供
一流的可观测性。

In [1]:
// process.env.OPENAI_API_KEY = "sk_...";

// 可选，在 LangSmith 中添加跟踪
// process.env.LANGCHAIN_API_KEY = "ls__...";
process.env.LANGCHAIN_CALLBACKS_BACKGROUND = "true";
process.env.LANGCHAIN_TRACING_V2 = "true";
process.env.LANGCHAIN_PROJECT = "Time Travel: LangGraphJS";

Time Travel: LangGraphJS


## 定义状态

状态是图中所有节点的接口。


In [2]:
import { Annotation } from "@langchain/langgraph";
import { BaseMessage } from "@langchain/core/messages";

const StateAnnotation = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
  }),
});

## 设置工具

我们将首先定义我们想要使用的工具。对于这个简单的例子，我们将
使用创建占位符搜索引擎。然而，创建起来确实很容易
您自己的工具 - 请参阅文档
[此处](https://js.langchain.com/docs/how_to/custom_tools) 了解如何操作
那。


In [3]:
import { tool } from "@langchain/core/tools";
import { z } from "zod";

const searchTool = tool(async (_) => {
  // 这是实际实现的占位符
  return "Cold, with a low of 13 ℃";
}, {
  name: "search",
  description:
    "Use to surf the web, fetch current information, check the weather, and retrieve other information.",
  schema: z.object({
    query: z.string().describe("The query to use in your search."),
  }),
});

await searchTool.invoke({ query: "What's the weather like?" });

const tools = [searchTool];

我们现在可以将这些工具包装在一个简单的
[ToolNode](/langgraphjs/reference/classes/prebuilt.ToolNode.html)。
每当工具（函数）被调用时，该对象就会实际运行它们
我们的法学硕士。


In [4]:
import { ToolNode } from "@langchain/langgraph/prebuilt";

const toolNode = new ToolNode(tools);

## 设置模型

现在我们将加载
[聊天模型](https://js.langchain.com/docs/concepts/chat_models/)。

1.它应该与消息一起使用。我们将以表格形式代表所有代理状态
消息，因此它需要能够与它们很好地配合。
2.它应该与
[工具调用](https://js.langchain.com/docs/how_to/tool_calling/#passing-tools-to-llms),
这意味着它可以在响应中返回函数参数。

<div class="警告提示">
<p class="admonition-title">注意</p>
<p>
这些模型要求不是使用 LangGraph 的一般要求 - 它们只是这个示例的要求。
</p>
</div>

In [5]:
import { ChatOpenAI } from "@langchain/openai";

const model = new ChatOpenAI({ model: "gpt-4o" });

完成此操作后，我们应该确保模型知道它具有这些
可供调用的工具。我们可以通过调用来做到这一点
[bindTools](https://v01.api.js.langchain.com/classes/langchain_core_language_models_chat_models.BaseChatModel.html#bindTools)。


In [6]:
const boundModel = model.bindTools(tools);

## 定义图表

我们现在可以把它们放在一起。时间旅行需要一个检查点来保存
状态 - 否则你不会有任何东西去 `get` 或 `update`。我们将使用
这
[MemorySaver](/langgraphjs/reference/classes/index.MemorySaver.html),
它将检查点“保存”在内存中。

In [7]:
import { END, START, StateGraph } from "@langchain/langgraph";
import { AIMessage } from "@langchain/core/messages";
import { RunnableConfig } from "@langchain/core/runnables";
import { MemorySaver } from "@langchain/langgraph";

const routeMessage = (state: typeof StateAnnotation.State) => {
  const { messages } = state;
  const lastMessage = messages[messages.length - 1] as AIMessage;
  // 如果没有调用任何工具，我们就可以完成（响应用户）
  if (!lastMessage?.tool_calls?.length) {
    return END;
  }
  // 否则，如果有，我们继续并调用工具
  return "tools";
};

const callModel = async (
  state: typeof StateAnnotation.State,
  config?: RunnableConfig,
): Promise<Partial<typeof StateAnnotation.State>> => {
  const { messages } = state;
  const response = await boundModel.invoke(messages, config);
  return { messages: [response] };
};

const workflow = new StateGraph(StateAnnotation)
  .addNode("agent", callModel)
  .addNode("tools", toolNode)
  .addEdge(START, "agent")
  .addConditionalEdges("agent", routeMessage)
  .addEdge("tools", "agent");

// 这里我们只保存在内存中
let memory = new MemorySaver();
const graph = workflow.compile({ checkpointer: memory });

## 与代理交互

我们现在可以与代理进行交互。在交互之间你可以获取和更新
状态。

In [8]:

let config = { configurable: { thread_id: "conversation-num-1" } };
let inputs = { messages: [{ role: "user", content: "Hi I'm Jo." }] } as any;
for await (
  const { messages } of await graph.stream(inputs, {
    ...config,
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

Hi I'm Jo.
-----

Hello, Jo! How can I assist you today?
-----



请参阅此处运行的 LangSmith 示例
https://smith.langchain.com/public/b3feb09b-bcd2-4ad5-ad1d-414106148448/r

在这里你可以看到“代理”节点运行，然后我们的边缘返回`__end__`所以
该图在那里停止执行。

让我们检查当前的图形状态。

In [9]:
let checkpoint = await graph.getState(config);
checkpoint.values;

{
  messages: [
    { role: 'user', content: "Hi I'm Jo." },
    AIMessage {
      "id": "chatcmpl-A3FGf3k3QQo9q0QjT6Oc5h1XplkHr",
      "content": "Hello, Jo! How can I assist you today?",
      "additional_kwargs": {},
      "response_metadata": {
        "tokenUsage": {
          "completionTokens": 12,
          "promptTokens": 68,
          "totalTokens": 80
        },
        "finish_reason": "stop",
        "system_fingerprint": "fp_fde2829a40"
      },
      "tool_calls": [],
      "invalid_tool_calls": []
    }
  ]
}


当前状态是我们上面看到的两条消息，1.我们的HumanMessage
发送进来，2.我们从模型返回的AIMessage。

`next` 值为空，因为图形已终止（转换到
[[代码1]]）。

In [10]:
checkpoint.next;

[]


## 让我们让它执行一个工具

当我们再次调用该图时，它将在每个内部之后创建一个检查点
执行步骤。让我们让它运行一个工具，然后查看检查点。

In [11]:
inputs = { messages: [{ role: "user", content: "What's the weather like in SF currently?" }] } as any;
for await (
  const { messages } of await graph.stream(inputs, {
    ...config,
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

What's the weather like in SF currently?
-----



[
  {
    name: 'search',
    args: { query: 'current weather in San Francisco' },
    type: 'tool_call',
    id: 'call_ZtmtDOyEXDCnXDgowlit5dSd'
  }
]
-----

Cold, with a low of 13 ℃
-----

The current weather in San Francisco is cold, with a low of 13°C.
-----



请在此处查看上述执行的跟踪：
https://smith.langchain.com/public/0ef426fd-0da1-4c02-a50b-64ae1e68338e/r 我们可以
看到它计划了工具执行（即“代理”节点），然后“should_continue”
边缘返回“继续”，因此我们继续执行“操作”节点
工具，然后“代理”节点发出最终响应，这使得
“should_continue”边缘返回“end”。让我们看看如何更好地控制
这。

### 在工具之前暂停

如果您注意到下面的内容，我们现在将添加 `interruptBefore=["action"]` - 这意味着
在采取任何行动之前，我们先暂停一下。这是一个伟大的时刻，让
用户纠正并更新状态！当您想要拥有时，这非常有用
人在循环中验证（并可能改变）要采取的行动。

In [12]:
memory = new MemorySaver();
const graphWithInterrupt = workflow.compile({
  checkpointer: memory,
  interruptBefore: ["tools"],
});

inputs = { messages: [{ role: "user", content: "What's the weather like in SF currently?" }] } as any;
for await (
  const { messages } of await graphWithInterrupt.stream(inputs, {
    ...config,
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

What's the weather like in SF currently?
-----

[
  {
    name: 'search',
    args: { query: 'current weather in San Francisco' },
    type: 'tool_call',
    id: 'call_OsKnTv2psf879eeJ9vx5GeoY'
  }
]
-----



## 获取状态

您可以使用获取最新的图形检查点
[`getState(config)`](/langgraphjs/reference/classes/langgraph.Pregel.html#getState)。

In [13]:
let snapshot = await graphWithInterrupt.getState(config);
snapshot.next;

[ 'tools' ]


## 恢复
您可以通过使用 `null` 输入运行图形来恢复。检查点是
已加载，并且没有新输入，它将像没有发生中断一样执行。

In [14]:
for await (
  const { messages } of await graphWithInterrupt.stream(null, {
    ...snapshot.config,
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

Cold, with a low of 13 ℃
-----

Currently, it is cold in San Francisco, with a temperature around 13°C (55°F).
-----



## 查看完整历史记录

让我们浏览该线程的历史记录，从最新到最旧。


In [15]:
let toReplay;
const states = await graphWithInterrupt.getStateHistory(config);
for await (const state of states) {
  console.log(state);
  console.log("--");
  if (state.values?.messages?.length === 2) {
    toReplay = state;
  }
}
if (!toReplay) {
  throw new Error("No state to replay");
}

{
  values: {
    messages: [
      [Object],
      AIMessage {
        "id": "chatcmpl-A3FGhKzOZs0GYZ2yalNOCQZyPgbcp",
        "content": "",
        "additional_kwargs": {
          "tool_calls": [
            {
              "id": "call_OsKnTv2psf879eeJ9vx5GeoY",
              "type": "function",
              "function": "[Object]"
            }
          ]
        },
        "response_metadata": {
          "tokenUsage": {
            "completionTokens": 17,
            "promptTokens": 72,
            "totalTokens": 89
          },
          "finish_reason": "tool_calls",
          "system_fingerprint": "fp_fde2829a40"
        },
        "tool_calls": [
          {
            "name": "search",
            "args": {
              "query": "current weather in San Francisco"
            },
            "type": "tool_call",
            "id": "call_OsKnTv2psf879eeJ9vx5GeoY"
          }
        ],
        "invalid_tool_calls": []
      },
      ToolMessage {
        "content": "Cold, wi

## 重放过去的状态

要从这个地方重播，我们只需将其配置传递回代理即可。


In [16]:
for await (
  const { messages } of await graphWithInterrupt.stream(null, {
    ...toReplay.config,
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

Cold, with a low of 13 ℃
-----

The current weather in San Francisco is cold, with a low of 13°C.
-----



## 从过去的状态中分支出来

使用 LangGraph 的检查点，您可以做的不仅仅是重放过去的状态。
您可以分支以前的位置，让代理探索备用位置
轨迹或让用户“版本控制”工作流程中的更改。

#### 首先，更新之前的检查点

更新状态将通过将更新应用于
之前的检查点。让我们**添加一个工具消息**来模拟调用该工具。

In [17]:
const tool_calls =
  toReplay.values.messages[toReplay.values.messages.length - 1].tool_calls;
const branchConfig = await graphWithInterrupt.updateState(
  toReplay.config,
  {
    messages: [
      { role: "tool", content: "It's sunny out, with a high of 38 ℃.", tool_call_id: tool_calls[0].id },
    ],
  },
  // 更新的应用“就像”它们来自节点一样。默认情况下，
  // 更新将来自最后运行的节点。在我们的例子中，我们想要治疗
  // 此更新就好像它来自工具节点一样，因此下一个要运行的节点将是
  // 代理人。
  "tools",
);

const branchState = await graphWithInterrupt.getState(branchConfig);
console.log(branchState.values);
console.log(branchState.next);

{
  messages: [
    {
      role: 'user',
      content: "What's the weather like in SF currently?"
    },
    AIMessage {
      "id": "chatcmpl-A3FGhKzOZs0GYZ2yalNOCQZyPgbcp",
      "content": "",
      "additional_kwargs": {
        "tool_calls": [
          {
            "id": "call_OsKnTv2psf879eeJ9vx5GeoY",
            "type": "function",
            "function": "[Object]"
          }
        ]
      },
      "response_metadata": {
        "tokenUsage": {
          "completionTokens": 17,
          "promptTokens": 72,
          "totalTokens": 89
        },
        "finish_reason": "tool_calls",
        "system_fingerprint": "fp_fde2829a40"
      },
      "tool_calls": [
        {
          "name": "search",
          "args": {
            "query": "current weather in San Francisco"
          },
          "type": "tool_call",
          "id": "call_OsKnTv2psf879eeJ9vx5GeoY"
        }
      ],
      "invalid_tool_calls": []
    },
    {
      role: 'tool',
      content: "It's sunny 

#### 现在你可以从这个分支运行

只需使用更新的配置（包含新的检查点 ID）。轨迹
将跟随新分支。

In [18]:
for await (
  const { messages } of await graphWithInterrupt.stream(null, {
    ...branchConfig,
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

The current weather in San Francisco is sunny, with a high of 38°C.
-----

